# Gender Neutral Image Captioning

## Part I. Preparing Dataset for Training

In [1]:
from data_utils import get_activity_list, get_gender_nouns, get_qualified_dataset, get_training_data

In [2]:
annotations_path = './data/annotations/'
activity_image_ids = get_activity_list()
gender_nouns_lookup = get_gender_nouns()
captions_dict, im_gender_summary = get_qualified_dataset(annotations_path, gender_nouns_lookup, save_file = False)


Evaluating ground truth labels in train set

Caption 0 processed, out of 414113 captions
No. of qualified images processed: 0

Caption 100000 processed, out of 414113 captions
No. of qualified images processed: 6452

Caption 200000 processed, out of 414113 captions
No. of qualified images processed: 13359

Caption 300000 processed, out of 414113 captions
No. of qualified images processed: 24080

Caption 400000 processed, out of 414113 captions
No. of qualified images processed: 34712

Evaluating ground truth labels in val set

Caption 0 processed, out of 202654 captions
No. of qualified images processed: 35500

Caption 100000 processed, out of 202654 captions
No. of qualified images processed: 42016

Caption 200000 processed, out of 202654 captions
No. of qualified images processed: 52292


## Part II. Select Training Method

One of our motivation of the project is to counter the bias in the dataset. As ground truth labels are not availabie from the original COCO dataset, we are experimenting with different methods of balancing the dataset. In the **get_training_data** function in data_utils.py, there are 8 different modes of generating data.

    - random: randomized selection of qualified images
    - balanced_mode: balanced ratio between male, female and neutral
    - balanced_clean: balanced ratio between male, female and neutral, only use images when all captions agree on using the same gender
    - balanced_gender_only: same as balanced_mode, but without neutral captions
    - balanced_clean_noun: balanced ratio between male, female and neutral, only use images when all captions agree on using the same noun
    - clean_noun: only use images when all captions agree on the same noun
    - activity_balanced: from activity tagged image sets, choose same ratio of male, female, neutral image
    - activity_balanced_clean: similar to activity_balanced, but all captions must agree on the same gender
    
Note that it is possible that output size may be smaller than training_size, especially for activity_balanced and activity_balanced_clean. As for certain activities, the sample size of clean data might be limited for some classes, e.g. women wearing tie.

In [5]:
training_size = 1000

In [6]:
random = get_training_data(im_gender_summary, captions_dict, activity_image_ids, training_size = training_size, mode = "balanced_clean")


captions of 1000 images are added


## Part III. Train Model/ Load Model Weights

## Part IV. Predict on selected images